# Deep solutions task
### By Koren Gast

In this task I tried to develop a model that would learn to read and solve math word problems

In [1]:
import sys
sys.path.append("../src")
import pandas as pd
from utils import get_vocabulary, get_max,\
    get_varsAndEqn_str, pad_and_vectorize, to_wolfram_format
from models.models import EncoderDecoder_model
import load_data


C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load the data and convert the equations list to one str seperared by commas 

In [2]:
train_data = load_data.load_alldata()
train_data = get_varsAndEqn_str(train_data)
dev_data = pd.read_json("data/dev_data.json")
dev_data = get_varsAndEqn_str(dev_data)
test_data = pd.read_json("data/test_data.json")
test_data = get_varsAndEqn_str(test_data)


Combine the train and dev data to create vocabularies and determine max lengths

In [3]:
train_dev = train_data.append(dev_data, sort=False)

In [4]:
txt_length = int(get_max(train_dev, 'text')*1.5)
var_length = int(get_max(train_dev, 'str_vars')*1.5)
eqn_length = int(get_max(train_dev, 'str_eqn')*2)


In [5]:
txt_vocab = get_vocabulary(train_dev, is_text=True)
eqn_vocab = get_vocabulary(train_dev, is_text=False)



### Padding and vectorizing the text (X) and the equations (y) as follows:
#### For X:
Each text would be represented as an array in the length of txt_length. Then, each word from the original text would be represented as a number from the txt_vocab. No worsd has the number 0 as representation because 0 is for None. Text with length<txt_length would be padded with zeros.
#### For y:
y is an array in the length of: 2 + var_length + eqn_length.
The first elemnet of y is the number of vaiables, the next var_length elements are the variables (where 0 is none), then the next element is the number of equations and the last eqn_length elements are chars creating the equations 

In [6]:

train_data = pad_and_vectorize(train_data, txt_length, var_length,
                               eqn_length, txt_vocab, eqn_vocab)
dev_data = pad_and_vectorize(dev_data, txt_length, var_length,
                             eqn_length, txt_vocab, eqn_vocab)
test_data = pad_and_vectorize(test_data, txt_length, var_length,
                              eqn_length, txt_vocab, eqn_vocab)


In [7]:
input_shape = txt_length
output_shape = 2+var_length+eqn_length
txt_vocab_size = len(txt_vocab)
eqn_vocab_size = len(eqn_vocab)

### The model:
Neural networks preforms poorly when trying to preform basic arythmetics.
I decided to translate the text to a string of the equations and afterwords solving it using wolframalpha package. Therfore, I used a simple encoder-decoder model might being used to translate or summerize texts.


In [8]:
EPOCHS = 30
BATCH_SIZE = 50
model = EncoderDecoder_model(input_shape=input_shape,
                             output_shape=output_shape,
                             txt_vocab_size=txt_vocab_size,
                             eqn_vocab_size=eqn_vocab_size,
                             var_length=var_length,
                             eqn_vocab=eqn_vocab,
                             epochs=EPOCHS,
                             batch_size=BATCH_SIZE)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 160, 128)          962688    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 288, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 288, 128)          197120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 288, 53)           6837      
Total params: 1,429,813
Trainable params: 1,429,813
Non-trainable params: 0
_________________________________________________________________


In [9]:
fit_hist = model.fit(df=train_data, dev_data=dev_data)

Train on 3220 samples, validate on 374 samples
Epoch 1/30


  50/3220 [..............................] - ETA: 7:41 - loss: 3.9789 - acc: 0.0072

 100/3220 [..............................] - ETA: 5:54 - loss: 3.9432 - acc: 0.4519

 150/3220 [>.............................] - ETA: 5:13 - loss: 3.9037 - acc: 0.6019

 200/3220 [>.............................] - ETA: 4:51 - loss: 3.8557 - acc: 0.6802

...

3050/3220 [===========================>..] - ETA: 16s - loss: 0.5550 - acc: 0.9054

3100/3220 [===========================>..] - ETA: 11s - loss: 0.5549 - acc: 0.9054

3150/3220 [============================>.] - ETA: 6s - loss: 0.5547 - acc: 0.9054 

3200/3220 [============================>.] - ETA: 1s - loss: 0.5557 - acc: 0.9053

3220/3220 [==============================] - 319s 99ms/step - loss: 0.5553 - acc: 0.9053 - val_loss: 0.8915 - val_acc: 0.8788


### Results

In the allotted time, I could not creat a model generating solvable equations. Therfore, I can not examine the model using the answers for the equations. Instead I'm presenting a comparison between the real equations and the output of the model:

In [10]:
preds = model.predict(test_data)

preds['equations'] = preds.apply(to_wolfram_format, axis=1)
comparison = pd.DataFrame({
    'real_equations': test_data['equations'],
    'predicted_equations': preds['equations']})

In [12]:
print(comparison.head(100))

                                       real_equations predicted_equations
0             [unkn: x,y, equ: x-y=9, equ: 5*y=3*x+7]    [unkn: n, equ: ]
1             [unkn: x,y, equ: x-y=3, equ: 4*y-3*x=7]    [unkn: f, equ: ]
2   [unkn: x,y, equ: x = y + 9, equ: 7*y - 2*x = 157]    [unkn: n, equ: ]
3    [unkn: x,y, equ: x-y=(-38), equ: 2*x-5*y=(-217)]    [unkn: s, equ: ]
4   [unkn: x, y, equ: x = y - 4, equ: 2*x - 5*y = ...    [unkn: n, equ: ]
5                [unkn: x,y, equ: x+y=18, equ: x-y=4]    [unkn: s, equ: ]
6                [unkn: x,y, equ: x+y=22, equ: x-y=4]    [unkn: x, equ: ]
7               [unkn: x,y, equ: x-y=10, equ: x+y=34]    [unkn: x, equ: ]
8               [unkn: x,y, equ: x+y=38, equ: x-y=12]    [unkn: f, equ: ]
9                [unkn: x,y, equ: x+y=39, equ: x-y=7]    [unkn: s, equ: ]
10               [unkn: x,y, equ: x+y=20, equ: x-y=4]    [unkn: n, equ: ]
11               [unkn: x,y, equ: y-x=5, equ: x+y=29]    [unkn: n, equ: ]
12            [unkn: x,y, equ: x+y=621

## Future development:
1. Fitting with more epochs
2. Customize loss function:

	a. Pantelize less for mistaking with zeros (Nones)
	
	b. Pantelize more for mistaking with important chars (comma sign, equal sign)

3. Explore different architectures:
	
	a. Other than simple encoder-decoder
	
	b. Multiple models instead of one (i.e. one model for predicting how many equation		s, other for each equation length etc...)

4. Use dev set to tune hyperparameters (epochs, batch_size, #neurans in layer, #layers, etc... )

5. Use dev set to compare and choose between models
